## Distributing in Python

Ok, so you've heard about all the hype around distributed computing and want to get in on the goodness! Or maybe you're a distributed computing veteren, but still a python neophyte? Either way, this blog post may be just tht epython distributed computing primer that you need to get started!

I've recently needed to start diving into the wacky world of distributed computing in python. Typically I only use distributed computing for feature processing in production models. When you have tons of data streaming into your model and need to clean and process it quickly...well scala spark can do wonders! Of course there are plenty of other nice tools like Flink as well, but my collegues like scala spark so scala spark it is.

However, it's only recently that I've been getting involved with distributed computing in python. Mostly this has to do with research I'm doing around scaling stats models to rediculously large datasets and very complex processes. Unlike most of my blogs where I bury the lead a bit and try to paint a story (I love stories), I'm starting this blog with the punch line:

1. Python was not made to be distributed, so distributing in python is not as easy as in other languages
2. Not all distributing is made equally. 
    1. Handling IO for a smooth user inerface (i.e. in a web app): Use multi-threading
    2. Parallel processes on a single machine: Multi-processing with shared memory
    3. Parallel processes on multiple machines: Multi-processing without shared memory
    
All of these use cases can be handled by Dask, but Dask is a relatively new package and I've hit a few pot holes trying to use it. The first and second use cases can be handled by the Multiprocessing package. 

You can theoretically also use pyspark to do distributed computing in python, but I don't recommend it. Pyspark has given me and collegues many issues in the past. Dask is probably a better choice. 

Ok, let's start learning. Here is the blog overview:
* What is Multi-Threading?
* What is Multi-Processing?
* Dask 
* Closing Remarks

### What is Multi-Threading? 

In computing, a thread is a 'thread of execution'. Each thread represents a task. A process may contain multiple threads whcih all share the same resoruces (different processes do not share resources). Threads are light weight (they are the smallest unit of processing that can be performed on an OS) and have a very specific use. 

Since threads share resoruces, they are bad for dsitributing work acorss the CPU. If you have 24 virtual cores in your laptop, threading will only use the cores available to the process they are part of. Well that sucks! Why would we ever use threads? Well, imagine we were building a website and a user clicks on a button that uploads a picture and takes the user to a different aopge that displays that picture. It might take a while to upload the picture. If we try to go to the enw apge before the picture uploads all hell will break loose. With multithreading we can spin up a new thred to handle the upload and WAIT for this thread to complete before the original thread (handling cahnges between web pages) moves the user to the new page. 

It is for these use cases, where networking, data, and I/O (Input/Output) are important, that we might want to use multi-threading. 

Before we go too deep, let's pull out the code. Let's say we have a process that needs to download from four different data sources at once. None of the downloads depends on the others so we can run the independantly. each download takes 1 second. How long does this take to do on a single thread?

In [12]:
%%time

def fake_download(num):
    time.sleep(1)
    print("Download: {}".format(i))

for i in range(5):
    fake_download(i)

Downlaod: 0
Downlaod: 1
Downlaod: 2
Downlaod: 3
Downlaod: 4
CPU times: user 6.32 ms, sys: 2.42 ms, total: 8.74 ms
Wall time: 5.02 s


Ugh, 5 seconds? I ain't got time for that! Let's use multi-threading to speed this up.

In [13]:
import threading
from queue import Queue
import time

In [20]:
%%time

def fake_download(num):
    time.sleep(1)
    print("Download: {}".format(num))
    
for i in range(5):
    t = threading.Thread(target=fake_download, args=(i,))
    t.start()
    
t.join() # try commenting this out and then running

Download: 0
Download: 4
Download: 2
Download: 1
Download: 3
CPU times: user 3.94 ms, sys: 3.05 ms, total: 6.99 ms
Wall time: 1.01 s


Wow, it ran in one second(ish)! That's because we kick off each 'download' together rather than waiting for each one to finish. 

Of course, I'm trying to get the run time of the entire process. The only way I can get an accurate time, as it turns out, is to wait for the last process to finish. To do this I add `t.join()` at the end. This makes the main thread wait until the last thread completes to run. 

Try running the cell without the `t.join()`. You should see a time in milliseconds. That can't be right! The reason the time is so low is because the time fucntion in the main thread is closing out before download threads finish running. 

Sometimes it can be tedious managing threads using `threading`. We can use `Threadpool` from `multiprocessing` to abstract away some of the effort for us.

In [33]:
from multiprocessing.pool import ThreadPool

In [38]:
%%time 

def fake_download(num):
    time.sleep(1)
    print("Download: {}".format(num))

def Parallel(numbers, threads=1):
    pool = ThreadPool(threads)
    results = pool.map(fake_download, numbers)
    pool.close()
    pool.join()
    return results

numbers = [i for i in range(5)]
squaredNumbers = Parallel(numbers, 5)

Download: 0
Download: 4
Download: 2
Download: 3
Download: 1
CPU times: user 7.99 ms, sys: 4.91 ms, total: 12.9 ms
Wall time: 1.04 s


Ok, now let's talk about locks. We often want our functions to modify variables. Waht happens if multiple threads want to update the same variable? Locks are ways to ensure that only one thread can access a variable at a time. We "lock" the variable down until it is not being used. 

A properly used lock will ensure that when a fucntion pulls a variable, that variable will stay the same until the fucntion updates it. If the lock wasn't there a different function might pull or update the variable while the first function was still running. 

We won't dive any deeper into the details around multi-threading, but if you want to learn more, you can go [here](https://www.geeksforgeeks.org/multithreading-python-set-1/). 

It's important to note that multi-threading should be sued with caution. Multi-threading incurs overhead abd should only be used when needed. Try to avoid multi-threading for basic tasks. Multiple threads also increases the complexity of programs and makes them hard to debug. 

## What is Multi_processing?

Unlike many other languages, python has a Gloval Interpretor Lock (GIL). This means that python is not "thread-safe". Python objects have a globally enforced lock that prevents you from getting yourself into trouble. These saftey rails are nice for preventing you from shooting yourself in the foot in many cases, but limit our python process to using only one core of the CPU. 

Sometimes we want to use more than one core. Now certain versions of python do not have the GIL, like iron python an Jython. However, we usually aren't working in Jython so let's assume we need to work around the GIL to make use of multiple cores in our CPU. To use multiple cores we need to spin up multiple processes! Each process will have it's own GIL and memory.

I will warn you, while we are taking advantage of modern dat cpu architectures and leveraging multiple cores, there is a lot of overhead with multi-processing and it should only be used when needed. Many python pacakges (numpy for example) are very well optimized and can run quickly even without parallelisation. 

For our first example of multiprocessing we'll assume that each process is running on the same machine. Given this assumption, we may choose to share varaibles/memory. This places limits on how much we can scale, but it's quick and dirty and works for many use cases.

For fun I print out the job number and the id of the process that completed the job. The shared variable is `return dict`. The key in `return_dict` that prevents workers overwriting each other is the job number. Each job has a unique id in this case (what I'm calling the job number). 

In [39]:
from os import getpid
import multiprocessing

In [40]:
%%time
def worker(procnum, return_dict):
    '''worker function'''
    time.sleep(1)
    print( 'I am number %d in process %d' % (procnum, getpid()) )
    return_dict[procnum] = getpid()
    
manager = multiprocessing.Manager()
return_dict = manager.dict()

jobs = []
for i in range(5):
    p = multiprocessing.Process(target=worker, args=(i,return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()
print( return_dict.values() )

I am number 0 in process 5987
I am number 1 in process 5988
I am number 2 in process 5989
I am number 3 in process 5990
I am number 4 in process 5991
[5987, 5988, 5989, 5990, 5991]
CPU times: user 15 ms, sys: 18.3 ms, total: 33.4 ms
Wall time: 1.04 s


In this next example I do not share memory between processes. I also use a processing pool to define how many processes I want to spin up. In the example above I jsut spin up a process for each job, but I might have more jobs than I ahve cores on my CPU. In such as case I would set the number of processes to the number of idle cores. Processes would pick up jobs as they are available. 

In [45]:
%%time
def worker(procnum):
    time.sleep(1)
    print( 'I am number %d in process %d' % (procnum, getpid()) )
    return getpid()

pool = multiprocessing.Pool(processes = 5)
output = pool.map(worker, range(5))
pool.close()
pool.join()
print( output )

I am number 3 in process 6024
I am number 1 in process 6022
I am number 0 in process 6021
I am number 2 in process 6023
I am number 4 in process 6025
[6021, 6022, 6023, 6024, 6025]
CPU times: user 14.4 ms, sys: 20.1 ms, total: 34.6 ms
Wall time: 1.05 s


## Dask

Now you may want to distribute to more than one computer/server. For this use case (in python) I recommend Dask. You can also use pyspark if you so choose, but I find Dask to be easier to use in python.

Dask doesn't always play nice with jupyter. There can be some nasty curve balls depending on what port you run jupyter on and what environments you are using. I'll jsut leave the Dask code below. If it doesn't work with jupyter, I recommend running it in the command line or in a py file.

Run in terminal to avoid 'too many files' error. 
```python

import numpy as np
from dask.distributed import Client
from dask.distributed import as_completed
import functools
import time

def f(x):
    time.sleep(20)
    return i, np.random.normal(0,1,1)[0]

client = Client()  # set up local cluster on your laptop
wrapper = functools.partial(client.submit, f)

results = []
for i in range(5):
    results.append(wrapper(i))
results = as_completed(results)
for i in results:
    print(i.result())



```

## Closing Remarks

Hopefully you have some understanding of distributed computing in python now. It can be a powerful tool when used in the right way for the right kinds of problems. If you are looking for a good use case for trying out distributed computing, why not try to implement Hog Wild! training in python? 

Hog Wild is a method where each worker performs gradient descent on the weights of a model like lienar regression or a neural network. The weight values have no lock on them and so the workers can actually override each other's updates! THis is a very easy way to distribute training of a model and has something of a regularizing affect. Here is a nice blog post walking you through it: [Implementing Hogwild!](https://srome.github.io/Async-SGD-in-Python-Implementing-Hogwild!/)